# Finding the best time to post on HackerNews

We are looking at the dataset from Hacker News that involves user feedback to posts/questions or posts directly from Hacker News. We aim to see which category recives more interaction from users. 
We also seek to identify if posts created at a certain time recieve more comments than average.

In [1]:
from csv import reader
opened_file = open("hacker_news.csv")

read_file = reader(opened_file)
hn = list(read_file)
headers = hn[0]
hn = hn[1:]
hn[:5]


[['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01'],
 ['10301696',
  'Note by Note: The Making of Steinway L1037 (2007)',
  'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0',
  '8',
  '2',
  'walterbell',
  '9/30/2015 4:12']]

In [2]:
print(headers)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


In [3]:
ask_posts = []
show_posts = []
other_posts = []
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
print(len(ask_posts)) 
print(len(show_posts))
print(len(other_posts))


1744
1162
17194


In [4]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
    avg_ask_comments = int(total_ask_comments / len(ask_posts))
print(avg_ask_comments)

total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
    avg_show_comments = int(total_show_comments / len(show_posts))
print(avg_show_comments)

14
10


In order to be able to iterate through the lists created in the previous step, "ask_posts" and "show_posts", we first needed to change the previous code and append the entire row to each list rather than just the title.

If we appeneded the title, then in this step here when we call to convert the row[4] value into an integer we would get an error. This is because the ask_posts and show_posts would be just a list of strings, so row[4] would just be the 4th index value of the string. 
So once we append the entire row instead, we can call row[4] here which gives us the num_comments which we then can convert into an integer. 

As for the values, the ask_posts recieve more comments on average than the show posts at a ratio of 14:10.

In [5]:
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    result_list.append([created_at,num_comments])
    
    
counts_by_hour = {}
comments_by_hour = {}

date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    hour = row[0]
    comment_number = int(row[1])
    hour = dt.datetime.strptime(hour,date_format)
    row[0] = hour
    time = hour.strftime('%H')
    
    if time not in counts_by_hour:
        counts_by_hour[time] = 1
        comments_by_hour[time] = comment_number
    elif time in counts_by_hour:
        counts_by_hour[time] += 1
        comments_by_hour[time] += comment_number    

In the above cell: the goal was to create two dictionaries, one for the number of posts per hour (called counts_by_hour, and another for the comments per hour, called comments_by_hour).

We first used the ask_posts list from the previous step, and created a list of lists called result_list. This result list had lists within it that were taken from ask_posts and contain both the time/date at which the post was made and the number of comments associated with the post.

The next step was to create the two dictionaries. We wanted to look through the list of lists called result_list and extract the hour at which each post was made and the number of comments on the post. We first converted each list into a datetime format and then we were able to extract the time from that. 
Following this, we added each of these parameters to their corresponding dictionaries. 

In [6]:
avg_by_hour = []
for time in comments_by_hour:
    avg_by_hour.append([time,comments_by_hour[time]/counts_by_hour[time]])
print(avg_by_hour)    

[['20', 21.525], ['07', 7.852941176470588], ['03', 7.796296296296297], ['10', 13.440677966101696], ['14', 13.233644859813085], ['19', 10.8], ['12', 9.41095890410959], ['01', 11.383333333333333], ['21', 16.009174311926607], ['22', 6.746478873239437], ['16', 16.796296296296298], ['18', 13.20183486238532], ['13', 14.741176470588234], ['04', 7.170212765957447], ['11', 11.051724137931034], ['00', 8.127272727272727], ['09', 5.5777777777777775], ['15', 38.5948275862069], ['06', 9.022727272727273], ['05', 10.08695652173913], ['02', 23.810344827586206], ['17', 11.46], ['08', 10.25], ['23', 7.985294117647059]]


In the above, our goal was to simply identify the average number of comments per post.
We first create an empty dictionary named avg_by_hour. Then we loop through the comments_by_hour dictionary and for each hour, we take the corresponding key value that goes with that hour (AKA the number of comments in that hour) and calculate the average by dividing the key from the comments_by_hour by the key value from the count_by_hour dictionary with the same hour. 
This is displayed as a list of lists in avg_by_hour.

In [7]:
swap_avg_by_hour = []
for row in avg_by_hour:
    hour = row[0]
    avg = row[1]
    swap_avg_by_hour.append([avg,hour])


sorted_swap = sorted(swap_avg_by_hour,reverse=True)
print(sorted_swap)

[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [9.022727272727273, '06'], [8.127272727272727, '00'], [7.985294117647059, '23'], [7.852941176470588, '07'], [7.796296296296297, '03'], [7.170212765957447, '04'], [6.746478873239437, '22'], [5.5777777777777775, '09']]


In [8]:
print("Top 5 Hours for Ask Posts Comments")
hour_format = "%H"
for row in sorted_swap[:5]:
    avg = row[0]
    hour = row[1]
    hour = dt.datetime.strptime(hour,hour_format)
    hour = hour.strftime("%H") 
    average = " At {}:00, {:.2f} average comments per post".format((int(hour)-1),avg)
    print(average)

Top 5 Hours for Ask Posts Comments
 At 14:00, 38.59 average comments per post
 At 1:00, 23.81 average comments per post
 At 19:00, 21.52 average comments per post
 At 15:00, 16.80 average comments per post
 At 20:00, 16.01 average comments per post


The Hours listed are in Eastern Time. We converted the time to central time in the last step by subtracting 1 hour.

Overall in this most recent step, we first wanted to swap the values in the avg_by_hour list from the previous step. This simply switched the positions of the two values in the list. 
We then sorted the list in descending order from the highest average comments per post.

We then iterated from the first five values from this sorted list (which were the highest averages) and listed the hour of day associated with these averages. Lastly we converted from the Eastern time zone to the central time zone for the hours by converting the hour value from a str to an int and then subracting by 1.

In conclusion, the hours that have the highest chance of receiving comments are 14:00 (2 pm) Central US time, 1:00 (1 am) Central US time, and 19:00 (7 pm) Central US time.

# Average points per post

In [9]:
ask_points = []
show_points = []
for row in ask_posts:
    num_points = int(row[3])
    ask_points.append(num_points)
    total_ask = sum(ask_points)
for row in show_posts:
    number_points = int(row[3])
    show_points.append(number_points)
    total_show = sum(show_points)

avg_show_points = total_show / len(show_posts)
avg_ask_points = total_ask / len(ask_posts)
print('\n')
print("The average points per show post is {:.2f}".format(avg_show_points))
print('\n')
print("The average points per ask post is {:.2f}".format(avg_ask_points))



The average points per show post is 27.56


The average points per ask post is 15.06


# Determining the times posts recieve more points

In [10]:
import datetime as dt
result_list = []
for row in hn:
    created_at = row[6]
    points = int(row[3])
    result_list.append([created_at,points])
    
points_by_hour = {}

date_format = "%m/%d/%Y %H:%M"

for row in result_list:
    hour = row[0]
    num_points = int(row[1])
    hour = dt.datetime.strptime(hour,date_format)
    row[0] = hour
    time = hour.strftime('%H')
    
    if time not in points_by_hour:
        points_by_hour[time] = 1
        points_by_hour[time] = num_points
    elif time in points_by_hour:
        points_by_hour[time] += 1
        points_by_hour[time] += num_points   

In [18]:
swap_points = []
for time in points_by_hour:
    swap_points.append([points_by_hour[time],time])
   
sorted_points = sorted(swap_points,reverse=True)


In [19]:
sorted_points

[[73600, '17'],
 [69904, '15'],
 [66112, '16'],
 [63810, '14'],
 [63668, '18'],
 [63170, '19'],
 [62999, '13'],
 [49534, '12'],
 [46765, '21'],
 [45238, '20'],
 [41320, '22'],
 [41061, '11'],
 [38214, '10'],
 [38038, '00'],
 [37952, '23'],
 [30292, '09'],
 [28441, '08'],
 [27447, '02'],
 [27290, '01'],
 [26823, '07'],
 [24707, '03'],
 [23850, '04'],
 [20495, '05'],
 [20297, '06']]

In [17]:
print("Top 5 times for most points on posts (Central Time)")
print('\n')
date_format = "%H"
for row in sorted_points[:5]:
    points = int(row[0])
    hour = row[1]
    avg_points = points / len(hn)
    hour = dt.datetime.strptime(hour,date_format)
    hour = hour.strftime("%H")
    print("At {}:00 each post received an average of {:.2f} points".format((int(hour)-1),avg_points))

Top 5 times for most points on posts (Central Time)


At 16:00 each post received an average of 3.66 points
At 14:00 each post received an average of 3.48 points
At 15:00 each post received an average of 3.29 points
At 13:00 each post received an average of 3.17 points
At 17:00 each post received an average of 3.17 points
